<a href="https://colab.research.google.com/github/Ramanarayanan/DeepLearningAndGenerativeAi/blob/main/Copy_of_LLAMA_RAG_CHAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_groq langchain-community unstructured InstructorEmbedding sentence_transformers langchain-huggingface install faiss-cpu langchain-community

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [ ]:
!pip install -U langchain-community pymupdf rapidocr-onnxruntime faiss-cpu langchain_groq langchain-huggingface

  Using cached langchain_community-0.3.13-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_groq-0.2.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 M

In [ ]:
import os
import pickle

from google.colab import files, userdata
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader

In [ ]:
class DocumentAnalyzer:
    def __init__(self, file_name):
        self.file_name = file_name
        self.llm = self.init_llm()
        self.embeddings = self.init_embeddings()
        self.vector_index = self.load_or_create_vector_index()
        self.chain = self.init_qa_chain()

    def init_llm(self):
        from langchain_groq import ChatGroq
        from google.colab import userdata
        return ChatGroq(
            temperature=0.9,
            max_tokens=500,
            groq_api_key=userdata.get('groq_api_key_valu'),
            model_name="llama-3.1-70b-versatile"
        )

    def init_embeddings(self):
        from langchain_huggingface import HuggingFaceEmbeddings
        return HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

    def load_or_create_vector_index(self):
        file_path = "vector_database_index.pkl"
        if os.path.exists(file_path):
            with open(file_path, "rb") as f:
                return pickle.load(f)
        else:
            loader = PyMuPDFLoader(self.file_name, extract_images=True)
            file_data = loader.load()
            text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=150)
            converted_docs = text_splitter.split_documents(file_data)
            docs = [self.remove_ws(doc) for doc in converted_docs]
            vector_index = FAISS.from_documents(docs, self.embeddings)
            with open(file_path, "wb") as f:
                pickle.dump(vector_index, f)
            return vector_index

    def remove_ws(self, file_document):
        text = file_document.page_content.replace('\n', '')
        file_document.page_content = text
        return file_document

    def init_qa_chain(self):
        from langchain.chains import RetrievalQAWithSourcesChain
        return RetrievalQAWithSourcesChain.from_llm(llm=self.llm, retriever=self.vector_index.as_retriever())

    def ask_question(self, query):
        return self.chain({"question": query})

In [ ]:
from google.colab import files

uploaded = files.upload()
if uploaded:
    file_name = list(uploaded.keys())[0]
    analyzer = DocumentAnalyzer(file_name)
    while True:
        user_question = input("Enter your question (or type 'exit'): ")
        if user_question.lower() == "exit":
            break
        answer = analyzer.ask_question(user_question)
        print(answer.get('answer'))
else:
    print("No file uploaded")

Saving accenture.pdf to accenture (2).pdf
Enter your question (or type 'exit'): show business look for fiscal 2025 in summary
For fiscal 2025, the company expects revenues to be $17.69 billion in the first quarter, with a 9% increase in U.S. dollars and 8% in local currency. The company also expects GAAP diluted EPS for fiscal 2025 to be in the range of $12.43 to $12.79, and operating cash flow to be in the range of $9.4 billion to $10.1 billion. Additionally, the company expects to return at least $8.3 billion in cash to shareholders through dividends and share repurchases.


Enter your question (or type 'exit'): show important financial results a point and in table format
Here are the important financial results in point and table format:

**Key Financial Results:**
* Net cash provided by (used in) financing activities: $2,751,748 (increase) 
* Net increase (decrease) in cash and cash equivalents: $3,301,586 (increase)
* Cash and cash equivalents at the end of the period: $8,306,055
